# Active thermographic inspection for aviation asstes

In [1]:
# Dependencies
# pip install opencv-python bagpy cvbridge3 ipdb

### Global 

In [2]:
import rosbag
import os
import tqdm
import numpy as np
from sensor_msgs import point_cloud2
import sensor_msgs
import cv2
from cv_bridge import CvBridge
import matplotlib.pyplot as plt
import ros2_numpy

In [3]:
# Directory containing the bag files
main_directory = "/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/"

# Create an output directory for saving overlayed images
output_directory = "/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/output/"
os.makedirs(output_directory, exist_ok=True)

## Read .bag thermal files

In [4]:
bag_files = []

for root, dirs, files in os.walk(main_directory):
    for file in files:
        if not file.startswith('imu_gps'): # exclude gps bag files
            if file.endswith(".bag"):
                bag_files.append(os.path.join(root, file))

print(bag_files)

for file in bag_files:

    bag             = rosbag.Bag(file)
    print (' - filename          : ', bag.filename)
    print (' - get_message_count : ', bag.get_message_count())
    print (' - start_time        : ', bag.get_start_time())
    print (' - end_time          : ', bag.get_end_time())
    print (' - size              : ', bag.size)
    print (' - mode              : ', bag.mode)
    print ('\n\n')

    ## Step2 - Method1 - Get all topics
    topics          = []
    print (' - get_message_count : ', bag.get_message_count())
    with tqdm.tqdm_notebook(total = bag.get_message_count()) as pbar:
        for idx, (topic, msg, t) in enumerate(bag.read_messages()):
            pbar.update(1)
            topics.append(topic)
    topics = np.array(topics)
    print (' - topics : ', np.unique(topics))

    ## Step2 - Method2 - Get all topics
    topics = [topic for topic in bag.get_type_and_topic_info().topics]
    for topic in sorted(topics):
        print (topic)

    ## Step3 - Print messages of a topic
    # for idx, (topic, msg, t) in enumerate(bag.read_messages()):
    for idx, (topic, msg, t) in enumerate(bag.read_messages(topics=['/velodyne_packets'])):
        print (topic, type(msg), t.to_time())

['/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/after/thermal_left_2023-08-31-14-53-51.bag', '/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/after/thermal_right_2023-08-31-14-53-51.bag']
 - filename          :  /Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/after/thermal_left_2023-08-31-14-53-51.bag
 - get_message_count :  352
 - start_time        :  1693508034.7076309
 - end_time          :  1693508060.4848423
 - size              :  230778409
 - mode              :  r



 - get_message_count :  352


/var/folders/df/rc0lymgs6nvd3r8bhd0wfb880000gn/T/ipykernel_66358/3457782773.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  with tqdm.tqdm_notebook(total = bag.get_message_count()) as pbar:


  0%|          | 0/352 [00:00<?, ?it/s]

 - topics :  ['/thermal_left/image']
/thermal_left/image
 - filename          :  /Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/after/thermal_right_2023-08-31-14-53-51.bag
 - get_message_count :  346
 - start_time        :  1693508034.864173
 - end_time          :  1693508060.4376736
 - size              :  226845181
 - mode              :  r



 - get_message_count :  346


  0%|          | 0/346 [00:00<?, ?it/s]

 - topics :  ['/thermal_right/image']
/thermal_right/image


In [5]:
def get_img_from_ros_image_msg(msg):
    """ Returns image as a numpy array. 
    Note: can be used with any topic of message type 'sensor_msgs/Image'
    """
    msg.__class__ = sensor_msgs.msg.Image
    return ros2_numpy.numpify(msg)

def process_bag(bag_file):
    bag = rosbag.Bag(bag_file, 'r')
    bridge = CvBridge()
    
    thermal_images_1 = []
    thermal_images_2 = []
    timestamps = []

    for topic, msg, t in bag.read_messages(topics=['/thermal_right/image', '/thermal_left/image']):
        if topic == '/thermal_right/image':
            thermal_images_1.append(get_img_from_ros_image_msg(msg))
        elif topic == '/thermal_left/image':
            thermal_images_2.append(get_img_from_ros_image_msg(msg))
        
        timestamps.append(t.to_sec())

    bag.close()

    # Check if there are matching timestamps
#     if len(set(timestamps)) != 1:
#         raise ValueError("Timestamps don't match between thermal image topics")

    # Compute the difference between thermal images
    difference_images = [np.abs(img1 - img2) for img1, img2 in zip(thermal_images_1, thermal_images_2)]

    return timestamps, difference_images

In [6]:
if __name__ == '__main__':
    
    for file in bag_files:
        bag = rosbag.Bag(file)

        print(f"Processing {bag}...")
        
        timestamps, difference_images = process_bag(file)

#         # Example: Display the difference images
#         for i, diff_img in enumerate(difference_images):
#             plt.imshow(diff_img, cmap='jet')
#             plt.title(f'Difference Thermal Image {i}')
#             plt.colorbar()
#             plt.show()

Processing path:        /Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/after/thermal_left_2023-08-31-14-53-51.bag
version:     2.0
duration:    25.8s
start:       Aug 31 2023 14:53:54.71 (1693508034.71)
end:         Aug 31 2023 14:54:20.48 (1693508060.48)
size:        220.1 MB
messages:    352
compression: none [176/176 chunks]
types:       sensor_msgs/Image [060021388200f6f0f447d0fcd9c64743]
topics:      /thermal_left/image   352 msgs @ 14.3 Hz : sensor_msgs/Image...
Processing path:        /Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/after/thermal_right_2023-08-31-14-53-51.bag
version:     2.0
duration:    25.6s
start:       Aug 31 2023 14:53:54.86 (1693508034.86)
end:         Aug 31 2023 14:54:20.44 (1693508060.44)
size:        216.3 MB
messages:    346
compression: none [173/173 chunks]
types:       sensor_msgs/Image [060021388200f6f0f447d0fcd9c64743]
topics:      /thermal_right/image   346 msgs @ 14.3 Hz : sensor_msgs/Image...


## Read .mp4 thermal files

In [7]:
mp4_files = []

prefixes = ("camera_0", "camera_1", "camera_2", "camera_3")
    
for root, dirs, files in os.walk(main_directory):
    for file in files:
        if not file.startswith(prefixes): # exclude back cameras (camera 2 and 3) and front (camera 0 and 1)
            if file.endswith(".mp4"):
                mp4_files.append(os.path.join(root, file))

print(mp4_files)

['/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/before/thermal_right_1693507917_466666676.mp4', '/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/before/thermal_left_1693507917_566666678.mp4', '/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/after/thermal_right_1693508034_633333346.mp4', '/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/after/thermal_left_1693508034_633333346.mp4']


## Stage 1: Frame Alignment

## Stage 2: Compute Frame by Frame Difference

## Stage 3: Apply a Threshold

## Stage 4: Create a Binary Mask

In [8]:
cap = cv2.VideoCapture('/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/before/thermal_right_1693507917_466666676.mp4')

fps = 30

# Define region of interest
# Read the first frame to determine frame dimensions
ret, frame = cap.read()
if not ret:
    print("Error: Cannot read video file.")
    exit()

# Create a window to display the video and select ROI
cv2.namedWindow('Select ROI')
roi = cv2.selectROI('Select ROI', frame)

# Extract the coordinates of the ROI
x, y, w, h = roi

# Loop through the video frames
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Crop the frame to the selected ROI
    roi_frame = frame[y:y+h, x:x+w]
    frame_height = frame[y:y+h]
    frame_width = frame[x:x+w]
    
    # Display the ROI frame
    cv2.imshow('ROI Frame', roi_frame)

    # Press 'q' to exit the loop
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Save the masked frames as a new video
out = cv2.VideoWriter('/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/output/before/mask_thermal_right_1693507917_466666676.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

out.write(masked_frame)
        
# Release resources
cap.release()
cv2.destroyAllWindows()


# # Apply to the original frame
# masked_frame = cv2.bitwise_and(frame, frame, mask=binary_mask)

# # Display the masked frame in real-time (for visualization)
# cv2.imshow('Masked Frame', masked_frame)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


# # Release capture
# out.release()
# cap.release()

error: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'VideoWriter'
> Overload resolution failed:
>  - Can't parse 'frameSize'. Sequence item with index 0 has a wrong type
>  - VideoWriter() missing required argument 'frameSize' (pos 5)
>  - VideoWriter() missing required argument 'params' (pos 5)
>  - VideoWriter() missing required argument 'frameSize' (pos 5)


In [ ]:
# for file in mp4_files:
#     cap = cv2.VideoCapture(file)
    
#     # Define region of interest
#     # Read the first frame to determine frame dimensions
#     ret, frame = cap.read()
#     if not ret:
#         print("Error: Cannot read video file.")
#         exit()

#     # Create a window to display the video and select ROI
#     cv2.namedWindow('Select ROI')
#     roi = cv2.selectROI('Select ROI', frame)

#     # Extract the coordinates of the ROI
#     x, y, w, h = roi
    
#     writerInit = False

#     # Loop through the video frames
#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             break

#         # Crop the frame to the selected ROI
#         roi_frame = frame[y:y+h, x:x+w]
        
#         if(not writerInit):
#             h,w,_ = roi_frame.shape
#             out = cv2.VideoWriter('output.avi', fourcc, 25, (w, h))
#             writerInit = True

#         # Display the ROI frame
#         cv2.imshow('ROI Frame', roi_frame)

#         # Press 'q' to exit the loop
#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break
    
#     # Process each frame
#     threshold_value = 50
#     max_value = 100
#     fps = 30
    
#     # Apply threshold
#     ret, binary_mask = cv2.threshold(frame, threshold_value, max_value, cv2.THRESH_BINARY)
    
#     # Filter or refine the mask
# #     kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
# #     binary_mask = cv2.erode(binary_mask, kernel, iterations=1)
# #     binary_mask = cv2.dilate(binary_mask, kernel, iterations=1)
    
#     # Apply to the original frame
#     masked_frame = cv2.bitwise_and(frame, frame, mask=binary_mask)
    
#     # Display the masked frame in real-time (for visualization)
# #     cv2.imshow('Masked Frame', masked_frame)
# #     cv2.waitKey(0)
# #     cv2.destroyAllWindows()

#     # Save the masked frames as a new video
# #     out = cv2.VideoWriter('/output/' + file + '.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))
    
    
#     out.write(masked_frame)

#     # Release capture
#     cap.release()
    

In [ ]:
# for file in mp4_files:
#     cap = cv2.VideoCapture(file)
    
    
    

## Stage 5: Visualizations

### .mp4

In [13]:
import os
import numpy as np
import cv2

threshold = 200
area_of_box = 700        # 3000 for img input
min_temp = 50            # in fahrenheit
font_scale_caution = 1   # 2 for img input
font_scale_temp = 0.7    # 1 for img input

def convert_to_temperature(pixel_avg):
    """
    Converts pixel value (mean) to temperature (fahrenheit) depending upon the camera hardware
    """
    return pixel_avg / 2.25


def process_frame(frame):

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    heatmap_gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    heatmap = cv2.applyColorMap(heatmap_gray, cv2.COLORMAP_HOT)

    # Binary threshold
    _, binary_thresh = cv2.threshold(heatmap_gray, threshold, 255, cv2.THRESH_BINARY)

    # Image opening: Erosion followed by dilation
    kernel = np.ones((3, 3), np.uint8)
    image_erosion = cv2.erode(binary_thresh, kernel, iterations=1)
    image_opening = cv2.dilate(image_erosion, kernel, iterations=1)

    # Get contours from the image obtained by opening operation
    contours, _ = cv2.findContours(image_opening, 1, 2)

    image_with_rectangles = np.copy(heatmap)

    for contour in contours:
        # rectangle over each contour
        x, y, w, h = cv2.boundingRect(contour)

        # Pass if the area of rectangle is not large enough
        if (w) * (h) < area_of_box:
            continue

        # Mask is boolean type of matrix.
        mask = np.zeros_like(heatmap_gray)
        cv2.drawContours(mask, contour, -1, 255, -1)

        # Mean of only those pixels which are in blocks and not the whole rectangle selected
        mean = convert_to_temperature(cv2.mean(heatmap_gray, mask=mask)[0])

        # Colors for rectangles and textmin_area
        temperature = round(mean, 2)
        color = (0, 255, 0) if temperature < min_temp else (
            255, 255, 127)

        # Callback function if the following condition is true
        if temperature >= min_temp:
            # Call back function here
            cv2.putText(image_with_rectangles, "High temperature", (35, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, font_scale_caution, color, 2, cv2.LINE_AA)

        # Draw rectangles for visualisation
        image_with_rectangles = cv2.rectangle(
            image_with_rectangles, (x, y), (x+w, y+h), color, 2)

        # Write temperature for each rectangle
        cv2.putText(image_with_rectangles, "{} F".format(temperature), (x, y),
                    cv2.FONT_HERSHEY_SIMPLEX, font_scale_temp, color, 2, cv2.LINE_AA)

    return image_with_rectangles

def main():
    """
    Main driver function
    """
    ## For Video Input
    video = cv2.VideoCapture(str(main_directory + '/after/camera_0_1693508037_718953776.mp4'))
    video_frames = []

    while True:
        ret, frame = video.read()

        if not ret:
            break

        # Process each frame
        frame = process_frame(frame)
        height, width, _ = frame.shape
        video_frames.append(frame)

        # Show the video as it is being processed in a window
        cv2.imshow('frame', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video.release()
    cv2.destroyAllWindows()

    # Save video to output
    size = (height, width)
    out = cv2.VideoWriter(str(main_directory + '/output/after/camera_0_1693508037_718953776.mp4'),cv2.VideoWriter_fourcc(*'MJPG'), 100, size)

    for i in range(len(video_frames)):
        out.write(video_frames[i])
    out.release()

    # img = cv2.imread(str(base_dir + 'input_image.jpg'))
    #
    # # Process the image
    # processed_img = process_frame(img)
    # height, width, _ = processed_img.shape
    # dim = (int(width * 0.5),int(height * 0.5))
    #
    # resized_img = cv2.resize(processed_img, dim, interpolation=cv2.INTER_AREA)
    # cv2.imwrite(str(base_dir + 'output_image.jpg'), resized_img)
    #
    # saved_img = cv2.imread(str(base_dir + 'output_image.jpg'))
    # cv2.imshow('output', saved_img)
    #
    # cv2.waitKey(0)


if __name__ == "__main__":
    main()

OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


## Stage 6: Post-Processing

In [ ]:
# import tqdm
# import rosbag # only python2.7 (http://wiki.ros.org/kinetic/Installation/Ubuntu)
# import numpy as np

# import cv2
# import matplotlib.pyplot as plt
# %matplotlib inline

# ## Step1 - Read .rosbag
# rosbag_filename = '/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/after/thermal_left_2023-08-31-14-53-51.bag'
# bag             = rosbag.Bag(rosbag_filename)

# ## Step2 - Read Compressed Images Packets
# print (' - start_time        : ', bag.get_start_time())
# print (' - end_time          : ', bag.get_end_time())

# topics = ['/thermal_left/image', '/thermal_right/image']
# with tqdm.tqdm_notebook(total = bag.get_message_count(topics)) as pbar:
#     for idx, (topic, msg, t) in enumerate(bag.read_messages(topics=topics)):
#         pbar.update(1)
#         print (topic, type(msg), t.to_time())
        
#         img = cv2.imdecode(np.fromstring(msg.data, np.uint8), cv2.IMREAD_COLOR)
#         plt.imshow(img)
#         filename = os.path.join('/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/output/thermal_left', str(t.to_time()) + ".jpg")
#         cv2.imwrite(filename, img)
#         if idx > 150:
#             break

In [ ]:
# # open cv
# import cv2
# # plotting library
# import matplotlib.pyplot as plt


# video = cv2.VideoCapture(str(main_directory + '/after/thermal_right_1693508034_633333346.mp4'))
# video_frames = []

# while True:
#     ret, frame = video.read()

#     if not ret:
#         break

#     # Process each frame
#     frame = process_frame(frame)
#     height, width, _ = frame.shape
#     video_frames.append(frame)

#     # Show the video as it is being processed in a window
#     cv2.imshow('frame', frame)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# video.release()
# cv2.destroyAllWindows()

# colormap_image = cv2.applyColorMap(image, cv2.COLORMAP_TWILIGHT_SHIFTED)

# plt.figure()
# plt.imshow(colormap_image)
# plt.show()

In [ ]:
# import os
# import cv2
# import numpy as np
# import rosbag
# from sensor_msgs.msg import Image
# from cv_bridge import CvBridge

# # Initialize a CvBridge to convert ROS Image messages to OpenCV images
# bridge = CvBridge()

# # Directory containing the bag files
# main_directory = "/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/"

# # Create an output directory for saving overlayed images
# output_directory = "/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/output/"
# os.makedirs(output_directory, exist_ok=True)

# # Function to process a single bag file
# def process_bag_file(bag_file_path, main_directory, output_subdirectory):
#     # Open the bag file
#     bag = rosbag.Bag(bag_file_path)
    
#     # Iterate through messages in the bag file
#     for topic, msg, t in bag.read_messages(topics=['/thermal_right/image, /thermal_left/image']):
#         process_message(msg, output_directory, output_subdirectory)

#     # Close the bag file
#     bag.close()

# # Define a function to process each message in the bag file
# def process_message(msg, main_directory, output_subdirectory):
#     try:
#         # Convert the ROS Image message to an OpenCV image
#         cv_image = bridge.imgmsg_to_cv2(msg, desired_encoding="passthrough")

#         # Apply any necessary preprocessing to enhance thermal features
#         # For example, you can apply thresholding or other image processing techniques

#         # Create a mask to isolate the thermal region of interest
#         # Here, we assume the thermal region is in a specific temperature range
#         lower_temp_threshold = 30  # Adjust this threshold as needed
#         upper_temp_threshold = 100  # Adjust this threshold as needed
#         thermal_mask = cv2.inRange(cv_image, lower_temp_threshold, upper_temp_threshold)

#         # Apply the mask to the original image
#         masked_image = cv2.bitwise_and(cv_image, cv_image, mask=thermal_mask)

#         # Overlay the mask on the original image (for visualization)
#         overlay_image = cv2.addWeighted(cv_image, 0.7, masked_image, 0.3, 0)

#         # Save the overlayed image to the output subdirectory
#         ipdb.set_trace()
#         output_path = os.path.join(output_directory, output_subdirectory)
#         os.makedirs(output_path, exist_ok=True)
#         filename = os.path.splitext(os.path.basename(bag_file_path))[0] + "_overlayed.png"
#         output_file_path = os.path.join(output_path, filename)
#         cv2.imwrite(output_file_path, overlay_image)

#     except Exception as e:
#         print(e)

In [ ]:
# # Walk through the main directory and its subdirectories
# import ipdb

# for file in res:
#     bag = rosbag.Bag(file)
    
#     print(f"Processing {bag}...")
#     process_bag_file(file, main_directory, output_directory)


# # for root, dirs, files in os.walk(main_directory):
# #     for file in files:
# #         if not file.startswith('imu_gps'): # exclude GPS bag files
# #             if file.endswith(".bag"):
                
# # #                 ipdb.set_trace()
# #                 # Get the relative subdirectory structure
# #                 rel_subdirectory = os.path.relpath(root, main_directory)
# #                 output_subdirectory = os.path.join(rel_subdirectory, file)

# #                 print(f"Processing {bag_file_path}...")
# #                 process_bag_file(bag_file_path, output_subdirectory)


In [ ]:
# import os
# import cv2
# import numpy as np

# # Function to process and save masked images and masks separately
# def process_images(input_dir, output_dir):
    
#     # Iterate through each folder in the input directory
#     for folder_name in os.listdir(input_dir):
#         folder_path = os.path.join(input_dir, folder_name)
#         if not os.path.isdir(folder_path):
#             continue

#         # Create the corresponding output folders
#         masked_images_output = os.path.join(output_dir, folder_name, "masked_images")
#         masks_output = os.path.join(output_dir, folder_name, "masks")
#         os.makedirs(masked_images_output, exist_ok=True)
#         os.makedirs(masks_output, exist_ok=True)

#         # Iterate through images and masks in the folder
        
        
#         for filename in os.listdir(folder_path):
#             if filename.endswith('.bag'):
#                 image_path = os.path.join(folder_path, filename)
#                 mask_path = os.path.join(folder_path, filename.replace('.jpg', '_mask.jpg'))
                
#                 # Check if the mask file exists
#                 if os.path.exists(mask_path):
#                     # Load the thermal image and mask
#                     image = cv2.imread(image_path)
#                     mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

#                     # Apply the mask to the image
#                     masked_image = cv2.bitwise_and(image, image, mask=mask)

#                     # Save the masked image and mask separately
#                     masked_image_output_path = os.path.join(masked_images_output, filename)
#                     mask_output_path = os.path.join(masks_output, filename.replace('.jpg', '_mask.jpg'))

#                     cv2.imwrite(masked_image_output_path, masked_image)
#                     cv2.imwrite(mask_output_path, mask)

#                     print(f"Processed: {masked_image_output_path} and {mask_output_path}")

# # Define the input and output directories
# input_directory = "/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/"  # Replace with your input directory path
# output_directory = "/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/output/" # Replace with your output directory path

# # Call the process_images function
# process_images(input_directory, output_directory)


In [ ]:
# def extract_and_save_masks(bag_file, output_dir):
#     bag = rosbag.Bag(bag_file)
#     bridge = CvBridge()

#     # Create an output directory for each Bag file
#     bag_name = os.path.splitext(os.path.basename(bag_file))[0]
#     bag_output_dir = os.path.join(output_dir, bag_name, 'masks')
#     os.makedirs(bag_output_dir, exist_ok=True)

#     for topic, msg, _ in bag.read_messages():
#         if topic == '/thermal_camera/mask':
#             # Convert the ROS Image message to a NumPy array
#             mask = bridge.imgmsg_to_cv2(msg, desired_encoding="passthrough")

#             # Save the mask image
#             mask_filename = os.path.join(bag_output_dir, f'{msg.header.stamp}_mask.png')
#             cv2.imwrite(mask_filename, mask)
#             print(f'Saved mask: {mask_filename}')

#     bag.close()

In [ ]:
# main_directory = "/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/"
# output_directory = "/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/output/"

# # Process each Bag file in the specified directory
# for bag_filename in os.listdir(main_directory):
#     if bag_filename.endswith('.bag'):
#         bag_path = os.path.join(bag_directory, bag_filename)
#         extract_and_save_masks(bag_path, output_directory)